In [6]:
import json
import random
import pandas as pd

dataset = "Chest-X-ray"

# convert csv to jsonl
data = pd.read_csv(f"../../data/eval/{dataset}.csv")

# 将DataFrame转换为字典列表
data_list = data.to_dict(orient='records')

# 固定随机种子并打乱数据
random.seed(1234)
random.shuffle(data_list)

new_data = []
for idx, item in enumerate(data_list):
    # 提取img_path和Finding Label
    img_path = item.get("img_path")
    finding_label = item.get("Finding Label").lower()

    if isinstance(finding_label, str):
        finding_label = finding_label.replace('|', ', ')
        
    # 创建新的数据项
    new_item = {
        "image": img_path,
        "label": finding_label
    }

    # 添加到new_data列表中
    new_data.append(new_item)

# 将数据保存为JSONL文件
jsonl_file = f"../../data/eval/{dataset}.jsonl"
with open(jsonl_file, mode='w', encoding='utf-8') as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")

print(f"数据已成功保存到 {jsonl_file} 文件中。")

数据已成功保存到 ../../data/eval/Chest-X-ray.jsonl 文件中。


In [17]:
import json
import random
import pandas as pd

dataset = "Chest-X-ray"

data = [json.loads(line) for line in open(f"../../data/eval/{dataset}.jsonl")]


new_data = []
for idx, item in enumerate(data):
    new_item = {
        "image": item["image"],
        "text": "Analyze this medical image and output the category with the highest probability.", 
        "category": "conv",
        "label": item["label"],
        "question_id": f'{idx}-{item["label"]}',
    }
    new_data.append(new_item)

random.shuffle(new_data)

jsonl_file = f"../../data/eval/test_prompt/{dataset}_llava_val.jsonl"
with open(jsonl_file, mode='w', encoding='utf-8') as f:
    for item in new_data:
        f.write(json.dumps(item) + "\n")
        
print(f"数据已成功保存到 {jsonl_file} 文件中。")


# You are now acting as a knowledgeable radiologist. Please analyze the provided medical image and identify the most appropriate disease category or categories present in the patient. The diagnosis can involve one or more conditions. You must select the relevant categories from the following list: ['atelectasis', 'cardiomegaly', 'pleural effusion', 'infiltration', 'lung mass', 'lung nodule', 'pneumonia', 'pneumothorax', 'consolidation', 'edema', 'emphysema', 'fibrosis', 'pleural thicken', 'hernia', 'no finding']. Remember, you should only output the categories from the list, and no additional content.
## 只输出类别的prompt
# You are now acting as a knowledgeable radiologist. Please analyze the provided medical image and identify the most appropriate disease category or categories present in the patient. You can only output the corresponding index number of the disease in the list from 0 to 14 from the given list: ['atelectasis', 'cardiomegaly', 'pleural effusion', 'infiltration', 'lung mass', 'lung nodule', 'pneumonia', 'pneumothorax', 'consolidation', 'edema', 'emphysema', 'fibrosis', 'pleural thicken', 'hernia', 'no finding'].
##ABCD尝试 
# What type of disease is shown in this chest x-ray image? Choose one from A. Atelectasis.\n B. Cardiomegaly.\n C. Pleural Effusion.\nD. Infiltration.

数据已成功保存到 ../../data/eval/test_prompt/Chest-X-ray_llava_val.jsonl 文件中。


In [ ]:
import json
import re
import numpy as np

# 读取数据
output_path = 'path_to_your_output_file.json'
answers = [json.loads(line) for line in open(output_path)]

# 定义提取类别的正则表达式
category_pattern = re.compile(r"is\s+'(.+?)'\.")

# 初始化变量
correct_predictions = 0
total_predictions = len(answers)
error_count = 0
error_question_ids = []

# 遍历每个answer，提取labels和预测类别
for item in answers:
    # 获取标签（label），labels 可能包含多个标签，以逗号或其他符号分隔
    labels = ["-".join(item["question_id"].split("-")[1:])]  # 获取label
    labels = [label.lower() for label in labels]  # 转为小写以方便比较

    # 获取预测的text
    text = item["text"]

    # 尝试从text中提取预测的类别
    match = category_pattern.search(text)

    if match:
        predicted_category = match.group(1).lower()  # 将预测类别转为小写
    else:
        # 如果无法提取预测类别，统计为出错
        error_count += 1
        error_question_ids.append(item["question_id"])
        continue  # 跳过此项

    # 检查预测类别是否在labels列表中
    if any(predicted_category in label for label in labels):
        correct_predictions += 1
    else:
        # 如果预测错误，统计出错信息
        error_count += 1
        error_question_ids.append(item["question_id"])

# 计算准确率
accuracy = (correct_predictions / total_predictions) * 100

# 输出结果
print(f"Total labels: {total_predictions}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Number of errors: {error_count}")
print(f"Error question IDs: {error_question_ids}")